<a href="https://colab.research.google.com/github/ajeevansiva/stock_news_summarizer/blob/main/Automating_News_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate
!pip install bitsandbytes

In [11]:
from transformers import AutoTokenizer, AutoModel, pipeline, AutoModelForSeq2SeqLM
from transformers import LlamaForCausalLM
# Specify the name of the pre-trained model
model_name = 'daryl149/llama-2-7b-chat-hf'


# Load the pre-trained model and its tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(pretrained_model_name_or_path='daryl149/llama-2-7b-chat-hf', load_in_8bit=True, device_map={'': 0},)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
import requests
from bs4 import BeautifulSoup
import time

def scrape_news(url):
    try:
        # Define headers
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
        }

        # Send a GET request to the URL with headers
        r = requests.get(url, headers=headers)
        r.raise_for_status()  # Raise an error for bad status codes

        # Parse the HTML content of the page
        soup = BeautifulSoup(r.text, 'html.parser')

        # Find all paragraphs and filter out messages
        paragraphs = soup.find_all('p')
        filtered_paragraphs = [p.text.strip() for p in paragraphs if "Thank you for your patience." not in p.text.strip()]

        return filtered_paragraphs
    except Exception as e:
        print("Error occurred:", e)
        return None


url = "https://www.dailymirror.lk/breaking-news/Tourist-rescued-after-falling-down-Sri-Pada-precipice/108-279832" #Edit url here

news = scrape_news(url)
newsString = ' '.join(news)
if news:
    print("News headlines:")
    for headline in news:
        print("-", headline)
else:
    print("No news scraped.")


News headlines:
- 
- 
- Last Updated : 2024-03-31 13:38:00
- Daily FT
- Sunday Times
- Mirror Edu
- Tamil Mirror
- Lankadeepa
- Middleast Lankadeepa
- Ada
- Deshaya
- Life Online
- Hi Online
- HitAd
- TimesJobs
- E-Paper
- Home delivery
- Advertise with us
- Mobile Apps
- feedback
- Archive
- Print Ads
- Sun, 31 Mar 2024 Today's Paper
- 29 March 2024 03:14 pm
  - 11      - {{hitsCtrl.values.hits}}
- By Ranjith Rajapakshe
- Ratnapura, March 29 (Daily Mirror)- An Indian tourist who fell down a more than 100 metre precipice at the Ehela Kanuwa, Ratnapura-Sri Pada route in the early hours of this morning was rescued by a team from the Maskeliya Police Special Task Force (STF) camp.
- Police said the tourist Bharath Chandradas (25) from Mumbai Maharastra had been trekking to Sri Pada along the Ratnapura route with several others when he fell off the protective fence down the precipice at about 5.00 am today.
- On information from the Udamaluwa police post, the STF rushed to the scene and re

In [5]:
from transformers import pipeline

def summarize_text(input_text, model_name="Ajeevan123/NewsSummarization", max_chunk_len=510):
    summarization_pipeline = pipeline("summarization", model=model_name, tokenizer=model_name)


    summarized_output = summarization_pipeline(input_text, max_length=510, min_length=40, do_sample=False)

    return summarized_output

# Example input text
input_text = newsString

# Summarize the input text
summary = summarize_text(input_text)
print(summary)
sr = str(summary[0]['summary_text'])
print(type(summary))
# Print the summary
print("Summary:")
print(sr)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1353 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'Bharath Chandradas (25) from Mumbai Maharastra had been trekking to Sri Pada along the Ratnapura route with several others when he fell off the protective fence down the precipice at about 5.00 am today. On information from the Udamaluwa police post, the STF rushed to the scene and rescued the tourist and provided first aid. Later, he was carried to Nallathanniya and from there admitted to hospital.'}]
<class 'list'>
Summary:
Bharath Chandradas (25) from Mumbai Maharastra had been trekking to Sri Pada along the Ratnapura route with several others when he fell off the protective fence down the precipice at about 5.00 am today. On information from the Udamaluwa police post, the STF rushed to the scene and rescued the tourist and provided first aid. Later, he was carried to Nallathanniya and from there admitted to hospital.


In [6]:
from transformers import pipeline
sentiment = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [7]:
result = sentiment(sr)
summary_sentiment_info = "Based on the description and the sentiment analysis answer the below prompts: Summary: " + sr + ", Sentiment: " + result[0]['label'] + ", Score: " + str(result[0]['score'])

In [8]:
summary_sentiment_info = "Based on the description and the sentiment analysis answer the below prompts: Summary: " + sr + ", Sentiment: " + result[0]['label'] + ", Score: " + str(result[0]['score'])
print(summary_sentiment_info)

Based on the description and the sentiment analysis answer the below prompts: Summary: Bharath Chandradas (25) from Mumbai Maharastra had been trekking to Sri Pada along the Ratnapura route with several others when he fell off the protective fence down the precipice at about 5.00 am today. On information from the Udamaluwa police post, the STF rushed to the scene and rescued the tourist and provided first aid. Later, he was carried to Nallathanniya and from there admitted to hospital., Sentiment: NEGATIVE, Score: 0.9753184914588928


In [11]:
!pip install accelerate
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [12]:
# User's prompt
user_prompt = "From which country is this person"


# Combine user prompt and summary sentiment info
instruction = f"### Instruction:\n{summary_sentiment_info}\n{user_prompt}\n### Response:\n"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)

# Generate the response
result = pipe(instruction)

response = result[0]['generated_text'][len(instruction):]

print(response)

Based on the information provided in the prompt, the person's country of origin is India, specifically the state of Maharashtra. Therefore, the answer to the prompt is:
India
